In [4]:
! pip install langchain_community modelscope faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall

# 此脚本用于生成进行embedding model finetuning的数据集

In [ ]:
! rm -rf ./BAAI/bge-base-zh-v1.5  # 确保路径正确，避免误删
! rm -rf ./BAAI/bge-reranker-base

In [9]:
! modelscope download --model BAAI/bge-reranker-base --local_dir ./BAAI/bge-reranker-base

流式输出内容被截断，只能显示最后 5000 行内容。


















































































































































































































































































































































































































































































































































































































Processing 10 items:  80% 8.00/10.0 [00:21<00:11, 5.99s/it]

































































































































































































































































































































In [7]:
! modelscope download --model BAAI/bge-base-zh-v1.5 --local_dir ./BAAI/bge-base-zh-v1.5

/bin/bash: line 1: rm-rf: command not found

 _   .-')                _ .-') _     ('-.             .-')                              _ (`-.    ('-.
( '.( OO )_             ( (  OO) )  _(  OO)           ( OO ).                           ( (OO  ) _(  OO)
 ,--.   ,--.).-'),-----. \     .'_ (,------.,--.     (_)---\_)   .-----.  .-'),-----.  _.`     \(,------.
 |   `.'   |( OO'  .-.  ',`'--..._) |  .---'|  |.-') /    _ |   '  .--./ ( OO'  .-.  '(__...--'' |  .---'
 |         |/   |  | |  ||  |  \  ' |  |    |  | OO )\  :` `.   |  |('-. /   |  | |  | |  /  | | |  |
 |  |'.'|  |\_) |  |\|  ||  |   ' |(|  '--. |  |`-' | '..`''.) /_) |OO  )\_) |  |\|  | |  |_.' |(|  '--.
 |  |   |  |  \ |  | |  ||  |   / : |  .--'(|  '---.'.-._)   \ ||  |`-'|   \ |  | |  | |  .___.' |  .--'
 |  |   |  |   `'  '-'  '|  '--'  / |  `---.|      | \       /(_'  '--'\    `'  '-'  ' |  |      |  `---.
 `--'   `--'     `-----' `-------'  `------'`------'  `-----'    `-----'      `-----'  `--'      `------'

Processin

In [2]:
# 加载数据集，进行向量化嵌入

from langchain_core.documents.base import Document as Document  # LangChain的文档对象，用于统一格式
import json
from langchain.vectorstores import FAISS
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy


# 步骤1：读取JSONL文件，转换为LangChain的Document对象列表
def load_jsonl_to_documents(jsonl_path):
    """
    读取JSONL文件，将每行转换为LangChain的Document对象
    假设JSONL中包含"context"字段（文本内容）和其他元数据（如"id"、"source"）
    """
    documents = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            # 解析每行JSON
            data = json.loads(line.strip())
            # 提取文本内容（根据你的JSONL结构调整字段名，如"page_content"、"text"等）
            text = data.get("context", "")  # 核心：需要嵌入的文本
            if not text:
                continue  # 跳过无文本的条目
            # 提取元数据（可选，如来源、ID等，会和向量一起存储）
            metadata = {k: v for k, v in data.items() if k != "context"}  # 排除文本字段
            # 创建Document对象（LangChain统一格式，方便后续处理）
            doc = Document(page_content=text, metadata=metadata)
            documents.append(doc)
    print(f"从JSONL文件加载了 {len(documents)} 个文档")
    return documents




def init_embedding_model(embedding_model_name: str = "./BAAI/bge-base-zh-v1.5") -> SentenceTransformerEmbeddings:

    """初始化嵌入模型"""
    embedding_model = SentenceTransformerEmbeddings(
        model_name=embedding_model_name
    )
    print(f"✅ 嵌入模型加载成功！模型路径：{embedding_model_name}")
    return embedding_model


def create_vector_db(documents, embeddings, db_save_path="./vector_db"):
    """将文档和嵌入向量存入FAISS向量库，并保存到本地"""
    # 生成向量并创建向量库（自动为documents中的文本生成嵌入）
    db = FAISS.from_documents(documents, embeddings,distance_strategy=DistanceStrategy.COSINE)
    # 保存向量库到本地（后续可直接加载，无需重新生成）
    db.save_local(db_save_path)
    print(f"向量数据库已保存至 {db_save_path}，包含 {len(documents)} 个向量")
    return db


jsonl_path="/content/sample_data/final_datasets.jsonl"
documents = load_jsonl_to_documents(jsonl_path)
# 2. 初始化嵌入模型
embeddings = init_embedding_model()
# 3. 生成向量并创建向量库
db = create_vector_db(documents, embeddings)

从JSONL文件加载了 572 个文档


/tmp/ipython-input-912969645.py:39: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(


✅ 嵌入模型加载成功！模型路径：./BAAI/bge-base-zh-v1.5
向量数据库已保存至 ./vector_db，包含 572 个向量


In [3]:
retrieved_docs = db.similarity_search_with_score(query="哪些情形会导致职工不能被认定为工伤或视同工伤",k=10)
retrieved_docs

[(Document(id='6c6de9eb-f878-44b9-b252-848538241cd4', metadata={'question': '哪些情形会导致职工不能被认定为工伤或视同工伤？', 'answer': '故意犯罪、醉酒或者吸毒、自残或者自杀的情形会导致职工不能被认定为工伤或视同工伤。', 'source_doc': '电力安全网_工伤保险条例（第三章）', 'groundedness_score': 5, 'groundedness_eval': '上文第十六条明确列举了三种情形，即故意犯罪、醉酒或者吸毒、自残或者自杀，这些情形会导致职工不能被认定为工伤或视同工伤。这些信息直接回答了问题，没有歧义。因此，根据给定的上下文，可以清晰无误地回答问题。', 'relevance_score': 5, 'relevance_eval': '这个问题对于国网公司内部员工了解工会相关规章制度非常有用。它直接涉及到员工的权益保护，特别是当员工在工作中受伤时，了解哪些情况不被视为工伤或视同工伤，可以帮助员工更好地理解自身权益，以及在必要时采取适当的法律或行政措施来保护自己。因此，这个问题有助于提高员工对工会规章制度的认识和理解，对于维护员工的合法权益具有重要意义。', 'standalone_score': 5, 'standalone_eval': '该问题明确询问了哪些情形下职工不能被认定为工伤或视同工伤，问题本身清晰，不需要额外的上下文信息来理解其含义。虽然问题涉及工伤认定的法律或规定，但只要了解工伤认定的基本概念，即可理解问题所指内容。因此，该问题独立于特定的上下文环境，评分应较高。'}, page_content='第十六条 职工符合本条例第十四条、第十五条的规定，但是有下列情形之一的，不得认定为工伤或者视同工伤：\n\n(一)故意犯罪的;\n\n(二)醉酒或者吸毒的;\n\n(三)自残或者自杀的。\n\n第十七条 职工发生事故伤害或者按照职业病防治法规定被诊断、鉴定为职业病，所在单位应当自事故伤害发生之日或者被诊断、鉴定为职业病之日起 30 日内，向统筹地区社会保险行政部门提出工伤认定申请。遇有特殊情况，经报社会保险行政部门同意，申请时限可以适当延长。\n\n用人单位未按前款规定提出工伤认定申请的，工伤职工或者其近亲属、工会组织在事故伤害发生之日

In [4]:
retrieved_docs[0][0].page_content

'第十六条 职工符合本条例第十四条、第十五条的规定，但是有下列情形之一的，不得认定为工伤或者视同工伤：\n\n(一)故意犯罪的;\n\n(二)醉酒或者吸毒的;\n\n(三)自残或者自杀的。\n\n第十七条 职工发生事故伤害或者按照职业病防治法规定被诊断、鉴定为职业病，所在单位应当自事故伤害发生之日或者被诊断、鉴定为职业病之日起 30 日内，向统筹地区社会保险行政部门提出工伤认定申请。遇有特殊情况，经报社会保险行政部门同意，申请时限可以适当延长。\n\n用人单位未按前款规定提出工伤认定申请的，工伤职工或者其近亲属、工会组织在事故伤害发生之日或者被诊断、鉴定为职业病之日起1 年内，可以直接向用人单位所在地统筹地区社会保险行政部门提出工伤认定申请。\n\n按照本条第一款规定应当由省级社会保险行政部门进行工伤认定的事项，根据属地原则由用人单位所在地的设区的市级社会保险行政部门办理。\n\n用人单位未在本条第一款规定的时限内提交工伤认定申请，在此期间发生符合本条例规定的工伤待遇等有关费用由该用人单位负担。\n\n第十八条 提出工伤认定申请应当提交下列材料：\n\n(一)工伤认定申请表;\n\n(二)与用人单位存在劳动关系(包括事实劳动关系)的证明材料;'

In [11]:
#加载rerank model and sample positive/negative pairs for finetuning

from sentence_transformers import CrossEncoder
import random
import json

# 1. Load a pre-trained CrossEncoder model
reranker = CrossEncoder("BAAI/bge-reranker-base")
pos_num = 3
neg_num = 5

jsonl_path="/content/sample_data/final_datasets.jsonl"
output_path = "/content/sample_data/final_datasets_for_embedding_finetuning.jsonl"

with open(jsonl_path, "r", encoding="utf-8") as f_in, open(output_path, "w", encoding="utf-8") as f_out:
        for line in f_in:
            # 解析每行JSON
            data = json.loads(line.strip())
            # 提取文本内容（根据你的JSONL结构调整字段名，如"page_content"、"text"等）
            query = data.get("question", "")
            docs = db.similarity_search_with_score(query=query,k=10)
            #使用rerank模型对检索的文档进行排序
            # Access the Document object within each tuple
            reranked_results = reranker.rank(query, [doc[0].page_content for doc in docs])
            #排序之后，取前三作为正样本，从剩下的检索的文档中抽样5个作为负样本，更新jsonl数据集

            sorted_indices = [item["corpus_id"] for item in reranked_results]
            # Create a list of Document objects from the original docs based on reranked order
            reranked_docs = [docs[idx][0] for idx in sorted_indices]


            positive_docs = reranked_docs[:pos_num]  # 前3个作为正样本
            remaining_docs = reranked_docs[pos_num:]  # 剩余文档（用于负样本）

            if len(remaining_docs) >= neg_num:
                negative_docs = random.sample(remaining_docs, neg_num)
            else:
                negative_docs = remaining_docs
                print(f"警告：问题 '{query}' 仅能抽取 {len(negative_docs)} 个负样本（不足{neg_num}个）")

            data["positive_samples"] = [doc.page_content for doc in positive_docs]
            data["negative_samples"] = [doc.page_content for doc in negative_docs]


            # 5. 写入更新后的数据到新JSONL文件
            f_out.write(json.dumps(data, ensure_ascii=False) + "\n")

In [12]:
  import pandas as pd

  df = pd.read_json(output_path, lines=True, encoding="utf-8")
  df.head()

,context,question,answer,source_doc,groundedness_score,groundedness_eval,relevance_score,relevance_eval,standalone_score,standalone_eval,positive_samples,negative_samples
0,# （二）其他直属单位兼职培训员数量要求\n\n1.各单位市公司、下属分公司的每个部门各不少...,各单位市公司、下属分公司的每个部门至少需要多少名兼职培训员？,各单位市公司、下属分公司的每个部门至少需要1名兼职培训员。,培训相关要求.pdf,5,上文明确指出“各单位市公司、下属分公司的每个部门各不少于1人（1-2人）”，这直接回答了问题...,3,该问题涉及国网公司内部的人员配置情况，特别是关于兼职培训员的数量要求，这与工会相关规章制度有...,4,该问题虽然指定了具体的组织结构（各单位市公司、下属分公司、每个部门），但并未提及特定的上下文...,[# （二）其他直属单位兼职培训员数量要求\n\n1.各单位市公司、下属分公司的每个部门各不...,[# 三、其他要求\n\n# （一）抓好全员常态化轮训及应急演练\n\n1.各单位在国康陕西...
1,# 第五章 正高级职称评定标准\n\n# 第十三条 基本要求\n\n（一）取得相应学历、学位...,正高级职称评定的基本要求有哪些？,取得相应学历、学位证书；取得主任记者或主任编辑职称证书，并满足相应专业工作年限；取得新闻记者证。,职称申报规范国网人才〔2025〕11号.pdf,5,上文提供了正高级职称评定的基本要求，包括学历学位证书、主任记者或主任编辑职称证书及相应的工作...,3,此问题关于正高级职称评定的基本要求，对于国网公司内部员工来说，了解职称评定的相关规定有助于他...,4,该问题询问的是正高级职称评定的基本要求，虽然涉及到特定的职业评定领域，但问题本身已经提供了足...,[# （三）正高级职称\n\n# 1．申报条件\n\n具备大学本科及以上学历（工业工程技术需...,[# 第二十三条 班主任经历要求\n\n申报技工院校教师系列职称的人员，应有在职业技术学校担...
2,# 1、个人所得税政策\n\n个人所得税以所得人为纳税人，以支付所得单位或者个人为扣缴义务人...,当全年的应纳税所得额超过660000元但不超过960000元时，个人所得税的适用税率是多少？,当全年的应纳税所得额超过660000元但不超过960000元时，个人所得税的适用税率是35%。,办公室（党委办公室）_岗位绩效工资管理实施方案及薪酬相关政策解读_20230716.pdf,5,根据给定的上下文，可以清晰、无歧义地找到当全年的应纳税所得额超过660000元但不超过960...,2,该问题主要涉及个人所得税的税率计算，与国网公司内部员工了解工会相关规章制度的直接关联性较低。...,5,该问题明确提出了一个具体的税务计算场景，即当个人的应纳税所得额落在660000元至96000...,[# 1、个人所得税政策\n\n个人所得税以所得人为纳税人，以支付所得单位或者个人为扣缴义务...,[# （二）支付标准\n\n1.范围内门诊费用：门诊特殊病种、门诊使用特殊药品扣除基本医疗保...
3,# 《国家电网有限公司安全教育培训工作规定》\n\n国网（安监/4）984-2019\n\n...,新入职人员的安全教育培训内容包括哪些方面？,新入职人员的安全教育培训内容主要包括：本单位安全生产情况；安全基本知识；安全生产规章制度、安...,员工规章制度应知应会手册.pdf,5,上文详细列出了新入职人员安全教育培训的内容，包括安全生产情况、安全基本知识、安全生产规章制度...,2,该问题主要关注的是新入职人员的安全教育培训内容，这与工会相关规章制度的直接关联性较低。工会规...,4,该问题询问的是新入职人员的安全教育培训内容，虽然它涉及特定的职业培训场景，但并不依赖于具体的...,[第二十五条各级调控机构应每年对所辖电网调度控制规程进行一次复查，根据需要进行补充修订；每三...,[（二）月、周、日安全生产例会。省公司级单位、地市公司级单位、县公司级单位应建立安全生产月、...
4,# 13.1一般注意事项\n\n13.1.1任何人进入生产现场（办公室、控制室、值班室和检修...,在检修期间需将栏杆拆除时，应采取什么措施？,应装设临时遮栏，并在检修结束时将栏杆立即装回。,电力安全工作规程_安全规程电缆部分.pdf,5,根据给定的上下文，可以清晰地了解到在检修期间需将栏杆拆除时，应采取的措施是装设临时遮栏，并在...,2,该问题主要涉及的是安全措施和操作规程，而不是直接与工会相关规章制度有关。虽然了解此类信息对于...,4,该问题描述了一个具体的场景，即“在检修期间需将栏杆拆除时”，并询问在这种情况下应采取什么措施...,[# 13.1一般注意事项\n\n13.1.1任何人进入生产现场（办公室、控制室、值班室和检...,[（二）严重缺陷：巡视人员应在1个工作日内将缺陷信息登录PMS系统提交班组长审核，并立即通知...


In [13]:
df.loc[0].positive_samples

['# （二）其他直属单位兼职培训员数量要求\n\n1.各单位市公司、下属分公司的每个部门各不少于1人（1-2人；\n\n2.各单位班组、变电站、项目部等若因工作原因，确实无法抽出人员参加紧急救护兼职培训员集中培训的，后续由所属市公司或分公司兼职培训员结合各项培训、会议等尽快为相关班组、变电站、项目部培训兼职培训员。\n\n# 三、其他要求',
 '# （二）其他直属单位兼职培训员数量要求\n\n1.各单位市公司、下属分公司的每个部门各不少于1人（1-2人）；\n\n2.各单位班组、变电站、项目部等若因工作原因，确实无法抽出人员参加紧急救护兼职培训员集中培训的，后续由所属市公司或分公司兼职培训员结合各项培训、会议等尽快为相关班组、变电站、项目部培训兼职培训员。\n\n# 三、其他要求',
 '# 培训相关要求\n\n# 一、各单位参培紧急救护兼职培训员基本条件\n\n1.冷静沉着，在紧急情况下能够保持镇定，迅速做出正确判断；\n\n2.身体素质好，能够应对培训中的体力要求，如高强度心肺复苏等；\n\n3.沟通能力强，具备持续学习意愿，愿意紧跟急救领域的最新发展学习更新相关知识；\n\n4.责任心强，愿意主动为职工群众提供热心服务。\n\n# 二、各单位参培紧急救护兼职培训员数量要求\n\n结合各单位机构实际情况，需满足从机关部门到班组这一最小作业单元均至少有1名兼职培训员。\n\n# （一）供电单位兼职培训员数量要求\n\n1.市、县公司本部的每个部门各不少于1人（1-2人）；2.各个工区下的每个部门各不少于1人（1-2人）；3.市、县公司部门及工区下辖有班组的，每个班组不少于1人（1-2人）；4.各供电所每个不少于1人（1-2人）；5.原集体企业各部门、下属分公司、班组每个1人。']

In [14]:
df.loc[0].negative_samples

['# 三、其他要求\n\n# （一）抓好全员常态化轮训及应急演练\n\n1.各单位在国康陕西公司完成兼职培训员集中培训后3个月内，组织本单位兼职培训员完成其他人员全员培训，并将本单位兼职培训员名单、全员培训名单签到及成绩反馈公司联系人，公司将不定期抽查，对未按时安排培训或培训效果较差的单位进行通报。\n\n2.各单位要建立内部常态化轮训及模拟演练机制，制定本单位应急预案并明确急救场景人员具体分工，突出“分工明确、灵活机动、快速反应”的特点，强化职工急救意识、配合能力和反应速度。结合各项工作和其他培训，统筹做好全员定期轮训和演练，原则上每年全员轮训、AED使用及紧急救护模拟演练各不少于1或2次。\n\n3.要严格抓好培训质量，坚决杜绝“走秀式”培训，以保障员工生命安全为使命，抓早、抓紧、抓实、抓细、抓好AED使用及紧急救护专题培训，助力全员在紧急情况发生时做到敢救、会救、\n\n能救。',
 '# 《国家电网有限公司教育培训管理规定》\n\n国网（人资/3）212-2019\n\n第十三条职工培训按类别分为入职培训、岗前培训、岗位培训和待岗培\n\n训等4类，相关规定如下：\n\n（一）人职培训。对新招录毕业生、复转军人等各类新入职人员，上岗前必须接受公司企业文化与战略、规章制度、安全生产、岗位知识和业务技能培训。培训时间和形式由用人单位根据岗位工作性质、人员素质基础等分别确定。用人单位须为新入职人员配备专门职业导师，签订师徒合同。\n\n（二）岗前培训。转岗、晋级等新上岗职工必须依据新岗位能力要求进行岗前培训并考试合格。国家实行准入制度的各类人员必须经过专门培训后持证上岗。\n\n（三）岗位培训。在岗职工必须根据岗位能力、技术进步、安全生产、保密规定等要求，定期进行培训。各类职工轮训周期不超过3年，3天及以上集中培训必须进行考试或考核。特殊岗位和人员岗位培训按照国家有关规定执行；各级领导人员及干部培训，按照国家及公司有关规定执行。\n\n（四）待岗培训。待岗职工必须接受待岗培训并考试合格，待岗培训不合格需进行再次培训的，培训有关费用由个人承担。',
 '# （二）建立兼职培训员动态管理机制\n\n1.各单位要建立兼职培训员动态管理机制，兼职培训员岗位变动时及时增补。若由于人员变动等原因导致相关部门、机构、班组兼职培训员空缺，需由所属市或县(分)公司组织其他兼职培训